In [17]:
import pandas as pd
import numpy as np
import DB_Utilities
DBTools = DB_Utilities.DBTools()  # instantiate the class
from  File_Utilities import FileTools
FileTools.MYDIR = ".\data"

redownload = False

In [ ]:
def get_file_by_year(year):
    if redownload:
        print(f"https://www2.census.gov/programs-surveys/cbp/datasets/{year}/cbp{year[2:]}co.zip")
        FileTools.unzip_file(FileTools.get_file_from_url(f"https://www2.census.gov/programs-surveys/cbp/datasets/{year}/cbp{year[2:]}co.zip"))

        print(f"https://www2.census.gov/programs-surveys/cbp/datasets/{year}/cbp{year[2:]}st.zip")
        FileTools.unzip_file(FileTools.get_file_from_url(f"https://www2.census.gov/programs-surveys/cbp/datasets/{year}/cbp{year[2:]}st.zip"))

        print(f"https://www2.census.gov/programs-surveys/cbp/datasets/{year}/cbp{year[2:]}us.zip")
        FileTools.unzip_file(FileTools.get_file_from_url(f"https://www2.census.gov/programs-surveys/cbp/datasets/{year}/cbp{year[2:]}us.zip"))

    # df_county = pd.read_csv(FileTools.get_full_file_path(f'cbp{year[2:]}co.txt'))
    df_county = FileTools.load_df_from_csv(f'cbp{year[2:]}co.txt')
    # df_state = pd.read_csv(FileTools.get_full_file_path(f'cbp{year[2:]}st.txt'))
    df_state = FileTools.load_df_from_csv(f'cbp{year[2:]}st.txt')
    # df_us = pd.read_csv(FileTools.get_full_file_path(f'cbp{year[2:]}us.txt'))
    df_us = FileTools.load_df_from_csv(f'cbp{year[2:]}us.txt')

    return df_county, df_state, df_us

# test
# df_county, df_state, df_us = get_file_by_year('2020')



In [ ]:
def convert_column_names_toLowerCase(df):
    df.columns = [x.lower() for x in df.columns]
    return df

def munge_data(_df):

    # they changed the case of the column names in 2016. Jerks.
    _df = convert_column_names_toLowerCase(_df)
    

    _df['naics'] = _df['naics'].str.replace('-', '')
    _df['naics'] = _df['naics'].str.replace(' ', '')
    _df['naics'] = _df['naics'].str.replace('/', '')
    _df['naics_level'] = _df['naics'].str.len()


    try:
        # If there is no county_fips just don't do this. 
        # This means that this data is coming from either the state or the us.
            # Get rid of counties with 999 fips, they are either statewide or unknown.
        _df = _df[_df['fipscty'] != 999].copy()
        _df['county_fips'] = _df['fipstate'].astype(str).str.zfill(2)+_df['fipscty'].astype(str).str.zfill(3)    
    except Exception as e:
        print(e)
        pass # hahaha. not handling this error today.
    
    return _df

In [ ]:
def run_all(year="2020", force_run=False):
   print(f"Running for year {year}")

   file_name = FileTools.get_full_file_path(f'cbp_emp_percent_by_county_state_us_{year}.gzip')   
   does_file_exist = FileTools.check_file(file_name)
   # if not then make it...
   if does_file_exist and not force_run:              
      print(f"Loading from file : {file_name} .")
      # emp_percent_by_county_state_us = pd.read_csv(file_name)
      emp_percent_by_county_state_us = FileTools.load_df_from_parquet(f'cbp_emp_percent_by_county_state_us_{year}.gzip')
   else:
      print(f"Making from scratch for : {year}")
      # Do all the work and save it.
      df_county, df_state, df_us =  get_file_by_year(year)

      print('Munging data')
      df_county = munge_data(df_county)
      # We only want top level employmnet data at the state and us level.
      # LFO             C       Legal Form of Organization

      #                         '-' - All Establishments                        
      #                         C - C-Corporations and other corporate legal forms of organization
      #                         Z - S-Corporations
      #                         S - Sole Proprietorships
      #                         P - Partnerships
      #                         N - Non-Profits
      #                         G - Government
      #                         O - Other

      df_state = munge_data(df_state)
      df_state = df_state[df_state.lfo == '-'] #Only All Establishments
      df_us = munge_data(df_us)
      df_us = df_us[df_us.lfo == '-'] #Only All Establishments


      print('Getting region level employment')
      # county level employment, we will join on this later as our base data
      df_county_emp = df_county[df_county.naics_level == 0][['county_fips','emp']]
      # state level employment, we need this to calculate the locaton quotient in relation to the state.
      df_state_emp = df_state[(df_state.naics_level == 0) ][['fipstate','emp']]
      # us level employment, we need this to calculate the locaton quotient in relation to the us.
      df_us_emp = df_us[(df_us.naics_level == 0) ][['uscode','emp']]


      print('Getting county level employment by industry')
      # Get the percent of each naics code in each county
      emp_percent_by_county = pd.merge(df_county, df_county_emp, on="county_fips")
      emp_percent_by_county['percent_of_county_emp'] = emp_percent_by_county['emp_x']/emp_percent_by_county['emp_y']
      emp_percent_by_county.rename(columns = {'emp_x':'emp_county_naics' , 'emp_y':'emp_county'}, inplace = True)

      print('Getting state level employment by industry')
      # Get the percent of each naics code in each state
      emp_percent_by_state = pd.merge(df_state, df_state_emp, on="fipstate")
      emp_percent_by_state['percent_of_state_emp'] = emp_percent_by_state['emp_x']/emp_percent_by_state['emp_y']
      emp_percent_by_state.rename(columns = {'emp_x':'emp_state_naics', 'emp_y':'emp_state' }, inplace = True)
      emp_percent_by_state = emp_percent_by_state[['fipstate','naics','percent_of_state_emp', 'emp_state_naics', 'emp_state']]

      print('Getting us level employment by industry')
      # Get the percent of each naics code in the US
      emp_percent_by_us = pd.merge(df_us, df_us_emp, on="uscode")
      emp_percent_by_us['percent_of_us_emp'] = emp_percent_by_us['emp_x']/emp_percent_by_us['emp_y']
      emp_percent_by_us.rename(columns = {'emp_x':'emp_us_naics', 'emp_y':'emp_us' }, inplace = True)
      emp_percent_by_us = emp_percent_by_us[['naics','percent_of_us_emp', 'emp_us_naics', 'emp_us']]
      emp_percent_by_us.head()

      print('Merging county with state and us')
      # Merge all the data together so we can calculate the location quotient.
      emp_percent_by_county_state = pd.merge(emp_percent_by_county, emp_percent_by_state, how='left', left_on=["naics", 'fipstate'], right_on=["naics", 'fipstate'])
      emp_percent_by_county_state_us = pd.merge(emp_percent_by_county_state, emp_percent_by_us, how='left', left_on=["naics"], right_on=["naics"])

      print('Calculating location quotient')
      # Calculate the location quotient for county/state and for county/us
      emp_percent_by_county_state_us['location_quotient_county_state'] = emp_percent_by_county_state_us['percent_of_county_emp']/emp_percent_by_county_state_us['percent_of_state_emp']
      emp_percent_by_county_state_us['location_quotient_county_us'] = emp_percent_by_county_state_us['percent_of_county_emp']/emp_percent_by_county_state_us['percent_of_us_emp']
      
      emp_percent_by_county_state_us.replace([np.inf, -np.inf], np.nan, inplace=True)
      emp_percent_by_county_state_us['year'] = year
#
      print('Saving to file')
      FileTools.save_df_as_parquet(emp_percent_by_county_state_us, f'cbp_emp_percent_by_county_state_us_{year}.gzip')
      print('Writing to dababase')
      # DBTools.truncate_and_insert_df(emp_percent_by_county_state_us, f"cbp_emp_percent_by_county_state_us_{year}")
      # DBTools.insert_df(emp_percent_by_county_state_us, f"cbp_emp_percent_by_county_state_us_{year}")


   return emp_percent_by_county_state_us

In [ ]:
years_to_get = [2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010]
# years_to_get = [2020]
Force_Run = False

dataframes = []
for year in years_to_get:
    dataframes.append(run_all(str(year), Force_Run))

all_years  = FileTools.concatenate_dataframes(dataframes)
all_years.shape

In [ ]:
naics_level = 2
# FileTools.save_df_as_parquet(all_years, f"cbp_lq_all_years.gzip")
all_years_naics  = all_years[all_years.naics_level == naics_level].copy()

# parquet is particular about mixed types, so we need to convert the objects to strings.
for df_column in all_years_naics.select_dtypes(include=['object']).columns:    
    all_years_naics [df_column] = all_years_naics [df_column].astype(str)

FileTools.save_df_as_parquet(all_years_naics , f"cbp_lq_{naics_level}digit_naics_all_years.gzip")

### Pivoting tables so county is row and all predictors are columns

In [ ]:
# Load in the data.
_cbp_lq_digit_naics_all_years_df = FileTools.load_df_from_parquet(f"cbp_lq_{naics_level}digit_naics_all_years.gzip")                                                                   

In [ ]:
all_data_cbp = pd.DataFrame()
for i in range(2010,2021): 
# for i in range(2020,2021): 
    year = str(i)
    print(f'Working on year : {year}')    

    # Filter by the year we want.
    temp_cbp_lq_digit_naics_all_years_df = _cbp_lq_digit_naics_all_years_df[
        (_cbp_lq_digit_naics_all_years_df['year'] == year) 
        & (_cbp_lq_digit_naics_all_years_df['naics_level'] == naics_level)
        ][[
            'county_fips','qp1', 'ap', 'est','naics','location_quotient_county_state','location_quotient_county_us']].copy()


    # Get some aggregations by county
    group_cbp_lq_digit_naics_all_years_df = temp_cbp_lq_digit_naics_all_years_df[['county_fips','qp1', 'ap', 'est']].groupby(['county_fips']).sum().reset_index()
    
    
    # Pivot all the naics codes up into columns
    pivot_cbp_lq_2digit_naics_all_years_df = pd.pivot_table(temp_cbp_lq_digit_naics_all_years_df, values='location_quotient_county_us', index=['county_fips'],
                        columns=['naics'], aggfunc=np.average, fill_value=0)


    # Add the aggregations to the pivot table
    merge_cbp_lq_2digit_naics_all_years_df = pivot_cbp_lq_2digit_naics_all_years_df.merge(group_cbp_lq_digit_naics_all_years_df, on='county_fips', how='left')
    merge_cbp_lq_2digit_naics_all_years_df['year'] = year

    # Do I need to do this?
    merge_cbp_lq_2digit_naics_all_years_df.reset_index(inplace=True)
    all_data_cbp = pd.concat([all_data_cbp, merge_cbp_lq_2digit_naics_all_years_df])


# Save the data to a parquet file.
FileTools.save_df_as_parquet(all_data_cbp, f'cbp_lq_{naics_level}digit_naics_all_years_PIVOT.gzip')
FileTools.save_df_as_csv(all_data_cbp, f'cbp_lq_{naics_level}digit_naics_all_years_PIVOT.csv') 

In [ ]:
# For two digit data

if(naics_level == 2):

    rename_dict = {'11':'Agriculture_Forestry_Fishing_Hunting',
    '21':'Mining_Quarrying_and_Oil_Gas_Extraction',
    '22':'Utilities',
    '23':'Construction',
    '31':'Manufacturing',
    '42':'Wholesale_Trade',
    '44':'Retail_Trade',
    '48':'Transportation_Warehousing',
    '51':'Information',
    '52':'Finance_Insurance',
    '53':'Real_Estate_Rental_Leasing',
    '54':'Professional_Scientific_and_Technical_Services',
    '55':'Management_of_Companies_Enterprises',
    '56':'Administrative_Support_Waste_Management_Remediation_Services',
    '61':'Educational_Services',
    '62':'Health_Care_Social_Assistance',
    '71':'Arts_Entertainment_and_Recreation',
    '72':'Accommodation_Food_Services',
    '81':'Other_Services_except_Public_Administration',
    '92':'Public_Administration'}

    all_data_cbp.rename(columns=rename_dict, inplace=True)  


    for i, k in enumerate(rename_dict):
        print(i, k)



    for i, k in enumerate(rename_dict):
        print(rename_dict[k])
        column_name = rename_dict[k]  
        try:  
            all_data_cbp[f'{column_name}_base'] = np.where(all_data_cbp[column_name] > 1, 1, 0)
        except:
            pass
        
    FileTools.save_df_as_parquet(all_data_cbp, f'cbp_lq_{naics_level}digit_naics_all_years_base_PIVOT.gzip')
    FileTools.save_df_as_csv(all_data_cbp, f'cbp_lq_{naics_level}digit_naics_all_years_base_PIVOT.csv') 


In [ ]:
test_all_data_cbp = FileTools.load_df_from_parquet(f'cbp_lq_{naics_level}digit_naics_all_years_base_PIVOT.gzip')
print(test_all_data_cbp.shape)

# DBTools.truncate_and_insert_df(all_data_cbp, "cbp_PIVOT")

In [ ]:
test_all_data_cbp.query("year == '2020'")

## Merge the data with patents

In [32]:
_patents = pd.read_csv('./data/patentsview/all_patents_granted_and pregrant.csv',dtype={'GEOID':'string', 'grant_year':'Int64', 'application_year':'Int64'}, low_memory=False)
_patents.GEOID = _patents.GEOID.str.rjust(5, '0')

_cbp_2_digit = pd.read_parquet(f'./data/cbp/cbp_lq_2digit_naics_all_years_base_PIVOT.gzip')
_cbp_2_digit = _cbp_2_digit.astype({'year': 'Int64'})


In [33]:
_patents[['GEOID', 'application_year']].dtypes

GEOID               string
application_year     Int64
dtype: object

In [34]:
_cbp_2_digit[['county_fips', 'year']].dtypes

county_fips    object
year            Int64
dtype: object

In [36]:
# Merge bea gdp and patents data
test_merge = pd.merge(_patents, _cbp_2_digit,
                      left_on = ['GEOID', 'application_year'],
                      right_on = ['county_fips', 'year'],
                      how = 'outer',
                      indicator = 'matched',
                      validate = 'many_to_one')

print(len(test_merge.query("matched == 'left_only'").GEOID.unique()))
# The missing values
missing_values_patents = pd.DataFrame(test_merge.query("matched == 'left_only'")).reset_index()

1881


In [45]:
# test_merge_no_terr = test_merge[~test_merge.GEOID.str.startswith(('78', '72' , '69','66' ))]
print(len(test_merge.query("matched == 'left_only' & application_year > 2009", engine='python').GEOID.unique()))
test_merge.query("matched == 'left_only' & application_year > 2009", engine='python').GEOID.unique()

32


array(['35028', '42057', '32029', '46021', '46102', '47127', '78030',
       '72127', '66010', '69110', '72025', '72053', '72021', '51075',
       '72091', '72001', '72023', '72103', '72077', '72061', '72031',
       '78010', '72069', '13289', '20031', '72113', '72097', '72133',
       '72153', '72139', '72027', '72117'], dtype=object)